<a href="https://colab.research.google.com/github/akondur18/Lexica/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [100]:
pip install evaluate

In [101]:
import numpy as np
import pandas as pd
import transformers #4.44.2 > 4.11 check
import os
os.environ["WANDB_DISABLED"]="true"
from datasets import load_dataset, Dataset, DatasetDict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [77]:
max_input_length = 128
max_target_length = 128

In [78]:
'''with open('train_english_small.txt', 'r', encoding='utf-8') as f:
    english_sentences = [line.strip() for line in f.readlines()]

with open('train_spanish_small.txt', 'r', encoding='utf-8') as f:
    spanish_sentences = [line.strip() for line in f.readlines()]''' #smaller dataset

with open('Expanded_GI_Care_Conversation_English.txt', 'r', encoding='utf-8') as f: #larger dataset
    english_sentences = [line.strip() for line in f.readlines()]

with open('Expanded_GI_Care_Conversation_Spanish.txt', 'r', encoding='utf-8') as f:
    spanish_sentences = [line.strip() for line in f.readlines()]

In [79]:
print(english_sentences[1])
print(spanish_sentences[1])

Good afternoon, Mrs. Martinez. I understand you’ve been having some digestive issues?
Buenas tardes, Sra. Martinez. ¿Entiendo que ha estado teniendo algunos problemas digestivos?


In [80]:
data = {'translation': [{'en': en, 'es': es} for en, es in zip(english_sentences, spanish_sentences)]}
train_df = pd.DataFrame(data)

In [81]:
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

datasets = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset
})
#create train and validation datasets

In [56]:
#metrics?

In [104]:
def preprocess(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["es"] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, text_target= targets, max_length= max_input_length, truncation= True) #create tokens from text
    return model_inputs

tokenized_datasets = datasets.map(preprocess, batched=True)

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [105]:
bs = 16
wd = 0.01
num_epoch = 3
#can change these values to get different models with different rates of loss

training_args = Seq2SeqTrainingArguments(
    output_dir= "./results",
    eval_strategy= "epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size= bs,
    per_device_eval_batch_size= bs,
    weight_decay= wd,
    save_total_limit= 3,
    num_train_epochs= num_epoch,
    predict_with_generate= True
)

#evaluation to be done at the end of each epoch, tweak the learning rate,
#bs is batch size --> change to see different outcomes
#wd is weight decay, can change to see different outcomes

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [106]:
trainer = Seq2SeqTrainer (
    model= model,
    args= training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset= tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

#for i in tqdm(x)

In [107]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.616457
2,No log,0.589533
3,No log,0.579777


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


TrainOutput(global_step=6, training_loss=0.533081849416097, metrics={'train_runtime': 86.6534, 'train_samples_per_second': 1.073, 'train_steps_per_second': 0.069, 'total_flos': 886389276672.0, 'train_loss': 0.533081849416097, 'epoch': 3.0})

In [108]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.579777181148529, 'eval_runtime': 1.0039, 'eval_samples_per_second': 7.969, 'eval_steps_per_second': 0.996, 'epoch': 3.0}


In [109]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-es' #before fine-tuning model
src_text = ["Let’s conduct an ECG to see how your heart is functioning and assess if we need to make any adjustments to your treatment."]

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
untuned = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

print(untuned)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['Vamos a conducir un ECG para ver cómo está funcionando su corazón y evaluar si necesitamos hacer algún ajuste a su tratamiento.']


In [66]:
!ls ./results/checkpoint-3

config.json		optimizer.pt   source.spm		tokenizer_config.json  vocab.json
generation_config.json	rng_state.pth  special_tokens_map.json	trainer_state.json
model.safetensors	scheduler.pt   target.spm		training_args.bin


In [110]:
from transformers import MarianMTModel, MarianTokenizer

model_name = './results/checkpoint-3'
#set model to new fine-tuned

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
src_text = ["Let’s conduct an ECG to see how your heart is functioning and assess if we need to make any adjustments to your treatment."]
inputs = tokenizer(src_text, return_tensors="pt", padding=True)

translated_tokens = model.generate(**inputs)

translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

print("After Fine-Tuning:", translated_text)

After Fine-Tuning: ['Llevemos a cabo un electrocardiograma para ver cómo está funcionando su corazón y evaluar si necesitamos hacer algún ajuste en su tratamiento.']


In [ ]:
#Reverse

In [90]:
with open('Expanded_GI_Care_Conversation_Spanish.txt', 'r', encoding='utf-8') as f:
    spanish_sentences2 = [line.strip() for line in f.readlines()]

with open('Expanded_GI_Care_Conversation_English.txt', 'r', encoding='utf-8') as f:
    english_sentences2 = [line.strip() for line in f.readlines()]

data2 = {'translation': [{'es': es, 'en': en} for es, en in zip(spanish_sentences2, english_sentences2)]}
df2 = pd.DataFrame(data2)

train_df2, test_df2 = train_test_split(df2, test_size=0.2, random_state=42)

train_dataset2 = Dataset.from_pandas(train_df2)
test_dataset2 = Dataset.from_pandas(test_df2)

datasets2 = DatasetDict({
    'train': train_dataset2,
    'test': test_dataset2
})

from transformers import MarianMTModel, MarianTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

model_name2 = 'Helsinki-NLP/opus-mt-es-en'
tokenizer2 = MarianTokenizer.from_pretrained(model_name2)
model2 = MarianMTModel.from_pretrained(model_name2)

def preprocess_function2(examples):
    inputs = [ex["es"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer2(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_datasets2 = datasets2.map(preprocess_function2, batched=True)

training_args2 = Seq2SeqTrainingArguments(
    output_dir="./spanish_to_english_results_2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs_2',
    logging_steps=10,
    save_steps=500,
)

data_collator2 = DataCollatorForSeq2Seq(tokenizer2, model=model2)

trainer2 = Seq2SeqTrainer(
    model=model2,
    args=training_args2,
    train_dataset=tokenized_datasets2['train'],
    eval_dataset=tokenized_datasets2['test'],
    tokenizer=tokenizer2,
    data_collator=data_collator2,
)

# Fine-tune the model
trainer2.train()
model2.save_pretrained("./spanish_to_english_results_2")
tokenizer2.save_pretrained("./spanish_to_english_results_2")

fine_tuned_model_name2 = "./spanish_to_english_results_2"

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,No log,0.996337
2,No log,0.974919
3,No log,0.964933


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


In [91]:
model_name2 = 'Helsinki-NLP/opus-mt-es-en' #before fine-tuning model
src_text2 = ["Vamos a conducir un ECG para ver cómo está funcionando su corazón y evaluar si necesitamos hacer algún ajuste a su tratamiento."]

tokenizer2 = MarianTokenizer.from_pretrained(model_name2)
model2 = MarianMTModel.from_pretrained(model_name2)

translated2 = model2.generate(**tokenizer2(src_text2, return_tensors="pt", padding=True))
output2 = [tokenizer2.decode(t, skip_special_tokens=True) for t in translated2]

print(output2)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['We will conduct an ECG to see how your heart is working and evaluate if we need to make any adjustments to your treatment.']


In [92]:
tokenizer2 = MarianTokenizer.from_pretrained(fine_tuned_model_name2)
model2 = MarianMTModel.from_pretrained(fine_tuned_model_name2)

spanish_text2 = ["Vamos a conducir un ECG para ver cómo está funcionando su corazón y evaluar si necesitamos hacer algún ajuste a su tratamiento."]

inputs2 = tokenizer2(spanish_text2, return_tensors="pt", padding=True)
translated_tokens2 = model2.generate(**inputs2)
translated_text2 = [tokenizer2.decode(t, skip_special_tokens=True) for t in translated_tokens2]

print("Translated Output:", translated_text2[0])

Translated Output: We will conduct an ECG to see how your heart is working and evaluate if we need to make any adjustments to your treatment.


Goal: Use translation API to